# tf-bert

## 1. Dataset

In [27]:
batch_size = 2

# **
# Create Dataset (Does the statement contains "cat" ?)
# *
texts_train = ['I like cat',
               'I do not like cat',
               'I like dog',
               'I do not like dog']
labels_train = [1,
                1,
                0,
                0]
texts_valid = ['I love cat',
               'I am cat',
               'I love dog',
               'I am dog']
labels_valid = [1,
                1,
                0,
                0]
texts_test = ['cat walked away from me',
              'I miss my dog']
labels_test = [1,
               0]

## 2. Model

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TFBertForSequenceClassification
CHECKPOINT = 'bert-base-uncased'

セルごとに Stand-alone で動作するように記述

仕様
* 出力はラベルではなく logits とする(loss の算出のため)

TODO

* 学習パイプラインのバリデーション（重み更新できてるのか、学習前後で比較し確認）
* データ、数件では無理があるのでちゃんとしたものに変更
* input・output を見直し End-to-end のネットワークに変更
  * input: Tensor\[str\] を受け取り tokenize・encoding する層に変更（WARNING 解消のため; ベストプラクティスなのかは要確認）
  * output: bert の logits を受け取り dense 等任意の層で受け取る実装に変更（テーブルデータとの concat 実装等、拡張する際必要）

### 2.1 素のモデル

In [116]:
# Data
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
train_encodings = tokenizer(texts_train, truncation=True, padding=True)
ds_train = tf.data.Dataset \
                        .from_tensor_slices((dict(train_encodings), labels_train)) \
                        .batch(2)
test_encodings = tokenizer(texts_test, truncation=True, padding=True)
ds_test = tf.data.Dataset \
                        .from_tensor_slices((dict(test_encodings))) \
                        .batch(2)

# Model
pretrained_model = TFBertForSequenceClassification.from_pretrained(CHECKPOINT)
# freeze
pretrained_model.trainable = False
# bert 最終層を trainable に変更
pretrained_model.bert.encoder.layer[-1].trainable = True
# 全結合層を trainable に変更
pretrained_model.layers[-1].trainable = True
pretrained_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=-0.0001),
              loss='binary_crossentropy',
              metrics='accuracy')

# Train
pretrained_model.fit(ds_train, epochs=100)

# Predict label
get_pred = lambda x: np.argmax(pretrained_model.predict(x)['logits'], axis=1)
get_pred(ds_test)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100
2/2 [==============================] - 6s 146ms/step - loss: 6.1093 - accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 0s 152ms/step - loss: 7.7125 - accuracy: 0.5000
Epoch 3/100
2/2 [==============================] - 0s 143ms/step - loss: 7.7125 - accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 0s 152ms/step - loss: 7.7125 - accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 140ms/step - loss: 6.0629 - accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 148ms/step - loss: 4.9080 - accuracy: 0.2500
Epoch 7/100
2/2 [==============================] - 0s 136ms/step - loss: 6.2857 - accuracy: 0.0000e+00
Epoch 8/100
2/2 [==============================] - 0s 144ms/step - loss: 6.0052 - accuracy: 1.0000
Epoch 9/100
2/2 [==============================] - 0s 137ms/step - loss: 4.8497 - accuracy: 0.5000
Epoch 10/100
2/2 [==============================] - 0s 138ms/step - loss: 7.7185 - accuracy: 0.7500
E

2/2 [==============================] - 0s 140ms/step - loss: 7.7125 - accuracy: 0.5000
Epoch 83/100
2/2 [==============================] - 0s 142ms/step - loss: 7.7125 - accuracy: 0.7500
Epoch 84/100
2/2 [==============================] - 0s 138ms/step - loss: 7.7176 - accuracy: 0.2500
Epoch 85/100
2/2 [==============================] - 0s 142ms/step - loss: 6.0811 - accuracy: 0.5000
Epoch 86/100
2/2 [==============================] - 0s 141ms/step - loss: 7.7258 - accuracy: 0.5000
Epoch 87/100
2/2 [==============================] - 0s 135ms/step - loss: 6.1789 - accuracy: 0.5000
Epoch 88/100
2/2 [==============================] - 0s 145ms/step - loss: 4.8022 - accuracy: 0.7500
Epoch 89/100
2/2 [==============================] - 0s 142ms/step - loss: 4.9943 - accuracy: 0.5000
Epoch 90/100
2/2 [==============================] - 0s 144ms/step - loss: 6.2059 - accuracy: 1.0000
Epoch 91/100
2/2 [==============================] - 0s 138ms/step - loss: 6.1162 - accuracy: 0.5000
Epoch 92/100


array([1, 0])

## 2.2 Sequential

WIP

In [21]:
# Data
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
train_encodings = tokenizer(texts_train, truncation=True, padding=True, return_tensors='tf')
ds_train = tf.data.Dataset \
                        .from_tensor_slices((dict(train_encodings), labels_train)) \
                        .batch(2)
test_encodings = tokenizer(texts_test, truncation=True, padding=True)
ds_test = tf.data.Dataset \
                        .from_tensor_slices((dict(test_encodings))) \
                        .batch(2)
### TODO: Data validation should be implemented

# Model
CHECKPOINT = 'bert-base-uncased'
pretrained_model = TFBertForSequenceClassification.from_pretrained(CHECKPOINT)
model = tf.keras.models.Sequential()
model.add(pretrained_model)
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics='accuracy')


# Train
model.fit(ds_train, epochs=2)

# Predict
model.predict(ds_test)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2


TypeError: in user code:

    File "/home/ubuntu/python3-venv/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/ubuntu/python3-venv/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ubuntu/python3-venv/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/ubuntu/python3-venv/lib/python3.8/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/ubuntu/python3-venv/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "<string>", line 3, in raise_from
        

    TypeError: Exception encountered when calling layer "sequential

```
WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor. Received: ...
```

Dataset の要素は tensor じゃないと遅くなってしまったりする?  
vectorizer_layer ならぬ encoding_layer を用意するべき?(受け取るのは Tensor\[str\])

## Bak

In [ ]:
# Construct
model = tf.keras.models.Sequential()
model.add(pretrained_model)
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# BERT は最終層のみ Fine tuning
model.layers[0].layers[0].trainable = False
model.layers[0].layers[1].trainable = False
model.layers[0].layers[2].trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics='accuracy')
# Train
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=4, verbose=0)
history = model.fit(ds_train,
                    epochs=2,
                    batch_size=batch_size,
                    validation_data=ds_valid,
                    callbacks=[early_stopping])

# # Learning curve
# pd.DataFrame({'train': history.history['loss'],
#               'valid': history.history['val_loss']}).plot()

# **
# Predict
# *
# for X, _ in ds_train:
#     print(model.predict_on_batch(X))